## Import Packages

In [ ]:
from math import floor

import pandas as pd

from rpy2.robjects.packages import importr # Import R Library
from rpy2.robjects import FloatVector # Converting data to R format
from rpy2.robjects import r # Calling R functions

importr('LongMemoryTS')

## Reading Data

In [ ]:
data = pd.read_csv('lndiff.csv')

In [ ]:
data = data.set_index(['year','iso']).drop('USA',level=1)

In [ ]:
def elw(data, delta=0.7,s2=False) -> dict:
    """
    Exact local Whittle estimator of the fractional difference parameter d
    for stationary and non-stationary long memory.
    """
    T = len(data)
    d = FloatVector(data)
    m = floor(1 + pow(T,delta))
    
    if s2==True:
        result = r['ELW2S'](d, m)
    else:
        result = r['ELW'](d, m)
    
    d_val = list(result.rx2('d'))[0]
    se_val = list(result.rx('s.e.')[0])[0]
    
    return {'d':d_val,'se':se_val}

In [ ]:
elw_df = data.groupby(level='iso')['rgdpmad'].apply(elw)

In [ ]:
elw2s_df = data.groupby(level='iso')['rgdpmad'].apply(elw,True)

In [ ]:
elwdata= pd.concat([elw_df,elw2s_df],axis=1)

In [ ]:
elwdata.columns = ['elw','elw2s']

In [ ]:
elwdata.to_csv('elwdata.csv')